<h1>Basic Test!</h1>

In [ ]:
!pip3 install requests

In [ ]:
import requests
import json 
#MY ELASTICSEARCH URL
URL = "http://127.0.0.1:9200/"

In [ ]:
#Request URL
R_URL = URL+"shakespeare/_source/1"

In [ ]:
# sending post request and saving response as response object 
r = requests.get(url = R_URL) 
  
# extracting response text  
response = r.text 
print("The response is:%s"%response) 

In [ ]:
y = json.loads(response)

In [ ]:
# the result is a Python dictionary:
print(y["play_name"]) 

In [ ]:
print(y)

In [ ]:
# pretty printing data
pretty_data = json.dumps(y, indent=4)
 
print(pretty_data)

<h1>Let's try a search request</h1>

In [ ]:
#Request URL
R_URL = URL+"shakespeare/_search"

In [ ]:
#PAYLOAD
R_data ="{\"query\": { \"match\" : { \"text_entry\" : {\"query\" : \"knife war edge\",\"operator\" : \"and\"}}} }"
JSON_DATA = y = json.loads(R_data)

In [ ]:
response = requests.post(url = R_URL, json = JSON_DATA)

In [ ]:
response  = response.text 
print("The response is:%s"%response) 

In [ ]:
#Convert to JSON
y = json.loads(response)
print(y)
# pretty printing data
pretty_data = json.dumps(y, indent=4)
 
print(pretty_data)

In [ ]:
#Print the specific field
print(y["hits"]["hits"][0]["_source"]["text_entry"])

<h1>Lercio Crawling</h1>

In [1]:
#Define requests object
import requests
import json 
#MY ELASTICSEARCH URL
URL = "http://127.0.0.1:9200/"

In [2]:
#New Lercio Index
index = "{ \"mappings\":{\"properties\":{\"title\" : {\"type\": \"text\" }, \"url\" : {\"type\": \"keyword\" },  \"data_articolo\" : { \"type\" : \"text\" }, \"abstract\" : { \"type\" : \"text\" }}}}"
JSON_DATA = json.loads(index)

#Request URL
R_URL = URL+"lercio"
RINS_URL = URL+"lercio/_doc/"

response = requests.put(url = R_URL, json = JSON_DATA)
response  = response.text 

#Convert to JSON
y = json.loads(response)
# pretty printing data
pretty_data = json.dumps(y, indent=4)
print(pretty_data)

{
    "acknowledged": true,
    "shards_acknowledged": true,
    "index": "lercio"
}


In [3]:
#import sys
#!{sys.executable} -m pip install scrapy

In [4]:
import scrapy
from scrapy.selector import Selector
from scrapy.crawler import CrawlerProcess

class LercioScraper(scrapy.Spider):
    name = "LercioPoliticalStories"
    start_urls = [
        'https://www.lercio.it/category/politica/',
    ]
    def parse(self, response):
        next_page = response.css('a.pagi')[-2].attrib['href']
        next_page_num = int(next_page.split('/')[-2])
        if next_page is not None and next_page_num <10:
           # print(next_page_num)
            yield scrapy.Request(next_page, callback=self.parse, dont_filter=True)
            
        
        #Select the news containers
        newses = response.css('.detailholder.medium').getall()
        for container in enumerate(newses):
            #print(container[1])
            #Get Title, Date, and Abstract
            news = Selector(text=container[1]).xpath('//h3/a')
            #print(news[0])
            n_link = news[0].attrib['href']
            title = Selector(text=news[0].get()).xpath('//text()').get().replace("\""," ").replace("'"," ")
            #print(title)
            dat = Selector(text=container[1]).xpath('//div/span/text()').get().replace("\""," ").replace("'"," ")
            #print(data)
            abstract = Selector(text=container[1]).xpath('//p/text()').get().replace("\""," ").replace("'"," ")
            #print(abstract)
            
            #Create json to send
            document = "{\"title\":\""+title+"\", \"url\":\""+n_link+"\",\"data_articolo\":\""+dat+"\",\"abstract\":\""+abstract+"\" }"
            JSON_DATA = json.loads(document)
            print(JSON_DATA)
            response = requests.post(url = RINS_URL, json = JSON_DATA)
            response  = response.text 

            #Convert to JSON
            y = json.loads(response)
            # pretty printing data
            pretty_data = json.dumps(y, indent=4)
            print(pretty_data)
           
process = CrawlerProcess()
process.crawl(LercioScraper)
process.start()

2019-07-18 19:17:05 [scrapy.utils.log] INFO: Scrapy 1.7.0 started (bot: scrapybot)
2019-07-18 19:17:05 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.1, Python 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.4.2, Platform Darwin-18.6.0-x86_64-i386-64bit
2019-07-18 19:17:05 [scrapy.crawler] INFO: Overridden settings: {}
2019-07-18 19:17:05 [scrapy.extensions.telnet] INFO: Telnet Password: 229ef0936ce4d6e9
2019-07-18 19:17:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-07-18 19:17:05 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.d

{'title': '“Noi della Lega”, lapsus freudiano di Di Maio nel suo ultimo post online', 'url': 'https://www.lercio.it/noi-della-lega-lapsus-freudiano-di-di-maio-nel-suo-ultimo-post-online/', 'data_articolo': 'Lug 09, 2019', 'abstract': 'Pomigliano d’Arco (NA) – Sembrava una normalissima giornata di riposo per il vice vice vice vice vice premier Luigi di Maio, in visita, con al'}
{
    "_index": "lercio",
    "_type": "_doc",
    "_id": "qf0WBmwB3j_6QniKYmlQ",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 0,
    "_primary_term": 1
}
{'title': 'Decreto Sicurezza Ter: le auto delle ONG in divieto di sosta verranno fatte saltare in aria', 'url': 'https://www.lercio.it/decreto-sicurezza-ter-le-auto-delle-ong-in-divieto-di-sosta-verranno-fatte-saltare-in-aria/', 'data_articolo': 'Lug 05, 2019', 'abstract': ' Salvini fascista, Salvini razzista, Salvini commercialista, Salvini densiometrista. Sa

2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 162
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 163
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 162
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 164


{
    "_index": "lercio",
    "_type": "_doc",
    "_id": "q_0WBmwB3j_6QniKY2kr",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 2,
    "_primary_term": 1
}
{'title': 'Ciclista sfiora volante della Polizia, Salvini: “Atto di guerra contro carro armato italiano”', 'url': 'https://www.lercio.it/ciclista-sfiora-volante-della-polizia-salvini-atto-di-guerra-contro-carro-armato-italiano/', 'data_articolo': 'Lug 03, 2019', 'abstract': 'Roma – “Ho già ordinato al ministro della Difesa di schierare al fronte tutti i combattenti di terra di mare e dell’aria, e di consegnare'}
{
    "_index": "lercio",
    "_type": "_doc",
    "_id": "rP0WBmwB3j_6QniKY2mi",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 3,
    "_primary_term": 1
}
{'title': 'È allarme per la nuova droga che ti fa credere di aver se

2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 164
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 165
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 165
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 164


...

{'title': 'Decreto Sicurezza: autovelox potranno sparare a chi supera i limiti di velocità', 'url': 'https://www.lercio.it/decreto-sicurezza-autovelox-potranno-sparare-a-chi-supera-i-limiti-di-velocita/', 'data_articolo': 'Dic 06, 2018', 'abstract': 'Gambizza (TO) – Il cosiddetto “Decreto Sicurezza“, firmato e fortemente voluto dal ministro dell’Interno Matteo Salvini, è stato approvato in via definitiva da Senato e'}
{
    "_index": "lercio",
    "_type": "_doc",
    "_id": "-f0WBmwB3j_6QniKeWn6",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 80,
    "_primary_term": 1
}
{'title': 'Di Maio precisa: “La qualifica di imprenditore onesto vale solo fino a dieci operai in nero”', 'url': 'https://www.lercio.it/di-maio-precisa-la-qualifica-di-imprenditore-onesto-vale-solo-fino-a-dieci-operai-in-nero/', 'data_articolo': 'Dic 05, 2018', 'abstract': 'Pomigliano D’Arco (Napul’è) – “Ho convocato 

2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 163
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 161
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 161
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 164
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1


{
    "_index": "lercio",
    "_type": "_doc",
    "_id": "_P0WBmwB3j_6QniKemmf",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 83,
    "_primary_term": 1
}
{'title': 'Trovato un iscritto del Pd che non si è ancora candidato alla segreteria del partito', 'url': 'https://www.lercio.it/trovato-un-iscritto-del-pd-che-non-si-e-ancora-candidato-alla-segreteria-del-partito/', 'data_articolo': 'Nov 27, 2018', 'abstract': 'Addaveni Baffo (NE) – Si chiama Milito Solamente, l’elettricista cinquantasettenne iscritto al Pd fin dalla sua fondazione nel 2008 e mai candidato alla segreteria del'}
{
    "_index": "lercio",
    "_type": "_doc",
    "_id": "_f0WBmwB3j_6QniKemnN",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 84,
    "_primary_term": 1
}
{'title': 'Solidarietà: Renzi diffama Travaglio pe

2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 163
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:17:12 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 163
2019-07-18 19:17:12 [scrapy.core.engine] INFO: Closing spider (finished)
2019-07-18 19:17:12 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 2583,
 'downloader/request_count': 9,
 'downloader/request_method_count/GET': 9,
 'downloader/response_bytes': 194548,
 'downloader/response_count': 9,
 'downloader/response_status_count/200': 9,
 'elapsed_time_seconds': 7.082128,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2019, 7, 18, 17, 17, 12, 539316),
 'log_count/DEBUG': 189,
 'log_count/INFO': 9,
 'memusage/max': 75194368,
 'memusage/startup': 75194368,
 'request_depth_max': 8,
 'response_received_count': 9,
 'scheduler/deq

{
    "_index": "lercio",
    "_type": "_doc",
    "_id": "Af0WBmwB3j_6QniKe2p-",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 88,
    "_primary_term": 1
}
{'title': 'Giuseppe Conte cerca la propria posizione con Google Maps ma il gps trova Salvini', 'url': 'https://www.lercio.it/giuseppe-conte-cerca-la-propria-posizione-con-google-maps-ma-il-gps-trova-salvini/', 'data_articolo': 'Nov 15, 2018', 'abstract': 'Augusto Rasori'}
{
    "_index": "lercio",
    "_type": "_doc",
    "_id": "Av0WBmwB3j_6QniKe2qv",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 89,
    "_primary_term": 1
}


In [5]:
#Let's do a search
#Request URL
R_URL = URL+"lercio/_search"

In [6]:
#PAYLOAD
R_data ="{\"query\": { \"match\" : { \"title\" : {\"query\" : \"Salvini lavoro\",\"operator\" : \"and\"}}} }"
JSON_DATA = y = json.loads(R_data)

In [7]:
response = requests.post(url = R_URL, json = JSON_DATA)
response  = response.text 

#Convert to JSON
y = json.loads(response)
# pretty printing data
pretty_data = json.dumps(y, indent=4)
print(pretty_data)

2019-07-18 19:19:46 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1
2019-07-18 19:19:46 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_search HTTP/1.1" 200 705


{
    "took": 1,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 2,
            "relation": "eq"
        },
        "max_score": 4.807936,
        "hits": [
            {
                "_index": "lercio",
                "_type": "_doc",
                "_id": "w_0WBmwB3j_6QniKaWnR",
                "_score": 4.807936,
                "_source": {
                    "title": "Tema: \u201cChe lavoro fa tuo pap\u00e0?\u201d Figlia di Salvini lascia il compito in bianco",
                    "url": "https://www.lercio.it/tema-che-lavoro-fa-tuo-papa-figlia-di-salvini-lascia-il-compito-in-bianco/",
                    "data_articolo": "Mag 01, 2019",
                    "abstract": "Pontida (BG) Una tranquilla giornata di propaganda\u00a0come le altre si \u00e8 tramutata in un inferno\u00a0tale da spiazzare\u00a0persino \u2018La Bestia\u2019, l\u201